# DFC compiler

This file is used to compile a onnx file into a hef file


## Onnx to har

In [5]:
# General imports used throughout the tutorial
import tensorflow as tf
from IPython.display import SVG
from pathlib import Path

modelPath = Path("models")
# import the ClientRunner class from the hailo_sdk_client package
from hailo_sdk_client import ClientRunner
import onnx
print(onnx.__version__)
chosen_hw_arch = "hailo8"

onnx_model_name = "RN50"
onnx_path = str(modelPath / f"{onnx_model_name}.onnx")

runner = ClientRunner(hw_arch=chosen_hw_arch)

hn, npz = runner.translate_onnx_model(
    onnx_path,
    onnx_model_name,
    # start_node_names=["modelInput"],
    # end_node_names=["modelOutput"],
    # disable_rt_metadata_extraction=True,
    # disable_shape_inference=True,
    # net_input_shapes={"modelInput": [1, 3, 224, 224]}
)

hailo_model_har_name = f"Harfiles/{onnx_model_name}_hailo_model.har"
runner.save_har(hailo_model_har_name)

1.14.0
[info] Translation started on ONNX model RN50
[info] Restored ONNX model RN50 (completion time: 00:00:00.39)
[info] Extracted ONNXRuntime meta-data for Hailo model (completion time: 00:00:01.80)
[info] Simplified ONNX model for a parsing retry attempt (completion time: 00:00:03.23)
[info] Start nodes mapped from original model: 'modelInput': 'RN50/input_layer1'.
[info] End nodes mapped from original model: '/attnpool/Squeeze'.
[info] Translation completed on ONNX model RN50 (completion time: 00:00:03.54)
[info] Saved HAR to: /home/lukasschoepf/Documents/ProjWork1_DFC/hailoDFC/Harfiles/RN50_hailo_model.har


### Visualization

In [6]:

!hailo visualizer {hailo_model_har_name} 

[info] Current Time: 12:56:27, 10/23/24
[info] CPU: Architecture: x86_64, Model: Intel(R) Core(TM) i7-14700, Number Of Cores: 28, Utilization: 2.6%
[info] Memory: Total: 31GB, Available: 22GB
[info] System info: OS: Linux, Kernel: 6.8.0-47-generic
[info] Hailo DFC Version: 3.29.0
[info] HailoRT Version: Not Installed
[info] PCIe: No Hailo PCIe device was found
[info] Running `hailo visualizer Harfiles/RN50_hailo_model.har`


## Har to har(optimized)

In [7]:
# General imports used throughout the tutorial
# file operations
import json
import os

import numpy as np
import tensorflow as tf
from IPython.display import SVG
from matplotlib import patches
from matplotlib import pyplot as plt
from PIL import Image
from tensorflow.python.eager.context import eager_mode
from pathlib import Path

# import the hailo sdk client relevant classes
from hailo_sdk_client import ClientRunner, InferenceContext

%matplotlib inline

# preprocessing
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize, PILToTensor
import torch

try:
    from torchvision.transforms import InterpolationMode
    BICUBIC = InterpolationMode.BICUBIC
except ImportError:
    BICUBIC = Image.BICUBIC

harPath = Path("Harfiles")
datafolder = Path("../../Data")
input_folder = datafolder / '02_data/hexagon_images/candolle_5patches'
calibFolder = datafolder / "calibData"



### Preprocess

For optimization a dataset is used.

In [8]:

def _convert_image_to_rgb(image):
    return image.convert("RGB")

def transform(n_px):
    """
    n_px: input resolution of the network
    """
    return Compose([
        Resize(n_px, interpolation=BICUBIC),
        CenterCrop(n_px),
        _convert_image_to_rgb,
        ToTensor(),
        # Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711)),
    ])

# First, we will prepare the calibration set. Resize the images to the correct size and crop them.
def preproc(image, output_height=224, output_width=224, resize_side=256):
    """imagenet-standard: aspect-preserving resize to 256px smaller-side, then central-crop to 224px"""
    with eager_mode():
        h, w = image.shape[0], image.shape[1]
        scale = tf.cond(tf.less(h, w), lambda: resize_side / h, lambda: resize_side / w)
        resized_image = tf.compat.v1.image.resize_bilinear(tf.expand_dims(image, 0), [int(h * scale), int(w * scale)])
        cropped_image = tf.compat.v1.image.resize_with_crop_or_pad(resized_image, output_height, output_width)

        return tf.squeeze(cropped_image)

preprocess = transform(224)
images_list = [img_name for img_name in os.listdir(input_folder) if os.path.splitext(img_name)[1] == ".jpg"]

calib_dataset = np.zeros((len(images_list), 224, 224, 3))
for idx, img_name in enumerate(sorted(images_list)):
    img = Image.open(os.path.join(input_folder, img_name))
    # img = PILToTensor(img)
    img_preproc = preprocess(img)
    img_transposed = np.transpose(img_preproc.numpy(),(1,2,0))
    calib_dataset[idx, :, :, :] = img_transposed

np.save(calibFolder / "calib_set.npy", calib_dataset)

In [9]:
# Second, we will load our parsed HAR from the Parsing Tutorial

model_name = "RN50"
hailo_model_har_name = f"{model_name}_hailo_model.har"
hailo_model_har_path = harPath / hailo_model_har_name
assert os.path.isfile(hailo_model_har_path), "Please provide valid path for HAR file"
runner = ClientRunner(har=str(hailo_model_har_path),hw_arch="hailo8")
# By default it uses the hw_arch that is saved on the HAR. For overriding, use the hw_arch flag.

In [10]:
# Now we will create a model script, that tells the compiler to add a normalization on the beginning
# of the model (that is why we didn't normalize the calibration set;
# Otherwise we would have to normalize it before using it)

# Batch size is 8 by default
# alls = "normalization1 = normalization([123.675, 116.28, 103.53], [58.395, 57.12, 57.375])\n" # From tutorial

# Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711)), # From Lia
alls = "normalization1 = normalization([0.48145466, 0.4578275, 0.40821073], [0.26862954, 0.26130258, 0.27577711])\n"

# Load the model script to ClientRunner so it will be considered on optimization
runner.load_model_script(alls)

# Call Optimize to perform the optimization process
runner.optimize(calib_dataset)

# Save the result state to a Quantized HAR file
quantized_model_har_path = f"{model_name}_quantized_model.har"
runner.save_har(quantized_model_har_path)

[info] Loading model script commands to RN50 from string
[info] Starting Model Optimization
[warning] Reducing optimization level to 0 (the accuracy won't be optimized and compression won't be used) because there's no available GPU
[warning] Running model optimization with zero level of optimization is not recommended for production use and might lead to suboptimal accuracy results
[info] Model received quantization params from the hn
[info] Starting Mixed Precision
[info] Mixed Precision is done (completion time is 00:00:00.28)
[info] LayerNorm Decomposition skipped
[info] Starting Statistics Collector
[info] Using dataset with 64 entries for calibration


Calibration:   0%|          | 0/64 [00:00<?, ?entries/s]2024-10-23 12:57:04.168466: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
2024-10-23 12:57:04.195620: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype float and shape [1,224,224,3]
	 [[{{node Placeholder/_0}}]]
Calibration:   2%|▏         | 1/64 [00:10<11:16, 10.74s/entries]2024-10-23 12:57:13.396950: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVAL

[info] Statistics Collector is done (completion time is 00:00:15.36)
[info] Starting Fix zp_comp Encoding
[info] Fix zp_comp Encoding is done (completion time is 00:00:00.00)
[info] Starting Matmul Equalization


ValueError: failed to initialize intent(inout) array -- expected elsize=8 but got 4

## Har(optimized) to hef


In [ ]:
from hailo_sdk_client import ClientRunner

model_name = "RN50x4"
quantized_model_har_path = modelPath / f"{model_name}_quantized_model.har"

runner = ClientRunner(har=quantized_model_har_path)
# By default it uses the hw_arch that is saved on the HAR. It is not recommended to change the hw_arch after Optimization.

hef = runner.compile()

file_name = f"{model_name}.hef"
with open(file_name, "wb") as f:
    f.write(hef)